In [1]:
#Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
pd.set_option('display.max_columns', None)

In [9]:
# Insert the names (or paths) of the tsv files
files = [
    "bombina/bombina_corset_DEGS__unk_not-unk.fasta.transdecoder.pep_nr.tsv",
    "bombina/bombina_corset_DEGS__unk_not-unk.fasta.transdecoder.pep_tr.tsv",
    "bombina/bombina_corset_DEGS__unk_not-unk.fasta.transdecoder.pep_sp.tsv"   
]

# Insert the titles of the graph
title = "bombina_pachypus_blastp"

# Insert the databases names
databases_names =[
    "Nr", 
    "TrEMBL",
    "Swiss-Prot",
]

table_path = "./bombina/bombina_unref_vs_not_unkref_table_padj_0.05----log2fc_1 (4).tsv"

# Insert the path of the graphs
path = "bombina/" + title

# Set the outformat
# e.g. 
# outfmt = "qseqid qlen sseqid sallseqid slen qstart qend sstart send qseq full_qseq sseq full_sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos qframe btop cigar staxids sscinames sskingdoms skingdoms sphylums stitle salltitles qcovhsp scovhsp qtitle qqual full_qqual qstrand"
# If there are column names in the file then set outfmt = None
outfmt = "qseqid qlen sseqid sallseqid slen qstart qend sstart send qseq full_qseq sseq full_sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos qframe btop cigar staxids sscinames sskingdoms skingdoms sphylums stitle salltitles qcovhsp scovhsp qtitle qqual full_qqual qstrand"

# Columns names
features = ["transcript", "row", "log2FoldChange", "padj", 
            "protein_accession", "sequence_identity", "alignment_length", 
            "evalue", "database", "gene", "locus_name", "sequence_description",
            "sequence_length", "organism", "protein_product"]

table = pd.read_csv(table_path, sep='\t')

In [5]:
table.head()

,transcript,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,NODE_100481_length_1662_cov_45.942320_g39073_i0,113.572774,3.729238,0.749879,4.973122,6.588303e-07,0.001008
1,NODE_100687_length_1658_cov_35.724701_g39182_i0,34.441283,2.713316,0.640121,4.238757,2.247607e-05,0.016458
2,NODE_100697_length_1658_cov_14.098680_g3928_i1,20.244586,-5.263078,1.052069,-5.002597,5.656316e-07,0.000894
3,NODE_101001_length_1653_cov_22.559899_g39352_i0,34.277619,2.555678,0.589651,4.334222,1.462765e-05,0.012317
4,NODE_101191_length_1650_cov_9.495262_g39455_i0,19.673311,-5.888866,1.467984,-4.011534,6.032561e-05,0.036529


In [6]:
def get_transcript(row):
    for t in table.transcript:
        if re.match(t, row):
            return t
    return None

df = pd.DataFrame()

for i in range(len(files)):

    #Import the dataset
    df_tmp = pd.read_csv(files[i], sep="\t", names=outfmt.split())

    df_tmp['transcript'] = df_tmp['qseqid'].apply(get_transcript)
    df_tmp['row'] = title
    df_tmp['sequence_identity'] = df_tmp.pident
    df_tmp['alignment_length'] = df_tmp.length
    df_tmp['evalue'] = df_tmp.evalue
    df_tmp['sequence_description'] = df_tmp.stitle
    df_tmp['sequence_length'] = df_tmp.slen
    df_tmp['database'] = databases_names[i]

    if "OS=" not in df_tmp.stitle[0]:    
        def get_sciname(x):
            
            os_index = - x[::-1].index('[')

            return x[os_index:-1]

        # Useful functions
        def get_protein_function(x):

            x_l = x.split(" ")

            return ' '.join(x_l[1:x_l.index(next(x for x in x_l if x.startswith('[')))])
        
        def get_protein_accession(x):
            return x.split(" ")[0]
        
        def get_locus_name(x):
            return None
        
        def get_gene(x):
            return None
    else:
        def get_sciname(x):

            os_index = x.index('OS=')
            ox_index = x.index('OX=')

            return x[os_index+3:ox_index-1]

        # Useful functions
        def get_protein_function(x):

            x_l = x.split(" ")

            return ' '.join(x_l[1:x_l.index(next(x for x in x_l if x.startswith('OS=')))])
        
        def get_protein_accession(x):
            return x.split("|")[1]
        
        def get_locus_name(x):
            return x.split("|")[2]
        
        def get_gene(x):

            try:
                gn_index = x.index('GN=')
                pe_index = x.index('PE=')
            except:
                return None
            return x[gn_index+3:pe_index-1]
        
    df_tmp['gene'] = df_tmp.stitle.apply(lambda x: get_gene(x))
    df_tmp['organism'] = df_tmp.stitle.apply(lambda x: get_sciname(x))
    df_tmp['protein_accession'] = df_tmp.sseqid.apply(lambda x: get_protein_accession(x))
    df_tmp['protein_product'] = df_tmp.stitle.apply(lambda x: get_protein_function(x))
    df_tmp['locus_name'] = df_tmp.sseqid.apply(lambda x: get_locus_name(x))

    df_tmp = pd.merge(df_tmp, table, on='transcript')

    df = pd.concat([df, df_tmp[features]])

df.sort_values(['transcript', 'evalue'], inplace=True)

df.reset_index(drop=True, inplace=True)

In [7]:
df.loc[df.duplicated(subset=['transcript', 'row', 'log2FoldChange', 'padj']), 'transcript':'padj'] = ''

In [14]:
df.to_excel(path + '.xlsx', index=False)